# GSM Arena Scraper
### Importing 

In [ ]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import spacy
from tqdm import tqdm
import numpy as np
import pandas as pd
from datetime import datetime
import time

### Scraping brand page links

In [20]:
url = 'https://www.gsmarena.com/makers.php3'
response = requests.get(url)
pg = response.content
bs_test = BeautifulSoup(pg, "html5lib")

In [21]:
m_l = []
t = bs_test.find_all("td")
for i in range(len(t)):
    m_l.append("https://www.gsmarena.com/" + str(t[i].find("a").get("href")))

In [22]:
m_l[:5]

['https://www.gsmarena.com/acer-phones-59.php',
 'https://www.gsmarena.com/alcatel-phones-5.php',
 'https://www.gsmarena.com/allview-phones-88.php',
 'https://www.gsmarena.com/amazon-phones-76.php',
 'https://www.gsmarena.com/amoi-phones-28.php']

### Brand page links with links to every page number

In [23]:
m_l_2 = []
for i in tqdm(m_l):
    url = i
    response = requests.get(url)
    pg = response.content
    bs_test = BeautifulSoup(pg, "html5lib")
    m_l_2.append(i)
    try:
        t = bs_test.find("div", {"class":"nav-pages"}).find_all("a")
    except:
        continue;
    for j in t:
        m_l_2.append("https://www.gsmarena.com/" + str(j.get("href")))

100%|██████████| 117/117 [01:06<00:00,  1.77it/s]


In [99]:
len(m_l_2)

318

### Mobile phone page links

In [30]:
m_l_3 = []
for i in tqdm(m_l_2):
    t = bs_test.find("div", {"class":"makers"}).find_all("li")
    for j in t:
        m_l_3.append("https://www.gsmarena.com/" + str(j.find("a").get("href")))

100%|██████████| 318/318 [00:00<00:00, 475.80it/s]


In [32]:
len(m_l_3)

12720

### Final dataframe with all features

In [92]:
brand = []
m_name = []
features = []
for i in tqdm(m_l_3):
    url = i
    response = requests.get(url)
    #time.sleep(3)
    pg = response.content
    bs_test = BeautifulSoup(pg, "html5lib")
    try:
        t = bs_test.find("h1", {"class":"specs-phone-name-title"}).text.split(" ")
    except:
        continue;
    if t[0] == "Fujitsu" or t[0] == "VK" or t[1] == "Ericsson":
        brand.append(" ".join(t[:2]))
        m_name.append(" ".join(t[2:]))
        continue;
    brand.append(" ".join(t[0:1]))
    m_name.append(" ".join(t[1:]))
    f = bs_test.find("div", {"id":"specs-list"}).find_all("tr")
    features_d = {}
    for j in f:
        try:
            heading = j.find("th")
            current_h = heading.text
            features_d[current_h] = {}
        except:
            pass
        try:
            f2 = j.find_all("td")
            features_d[current_h][f2[0].text] = f2[1].text
        except:
            pass
    features.append(features_d)

100%|██████████| 10/10 [00:07<00:00,  1.38it/s]


In [95]:
mobile_data = pd.DataFrame({"Brand":brand,"Mobile":m_name,"Features":features})

In [96]:
mobile_data.head()

,Brand,Mobile,Features
0,ZTE,Blade 20 Pro 5G,{'Network': {'Technology': 'GSM / CDMA / HSPA ...
1,ZTE,Blade V2020 5G,{'Network': {'Technology': 'GSM / CDMA / HSPA ...
2,ZTE,Watch Live,{'Network': {'Technology': 'No cellular connec...
3,ZTE,Blade 20 5G,{'Network': {'Technology': 'GSM / CDMA / HSPA ...
4,ZTE,Blade A7s 2020,"{'Network': {'Technology': 'GSM / HSPA / LTE',..."
